# COVID-19 Detection Inference

This notebook demonstrates how to use the trained COVID-19 detection model to make predictions on new X-ray images and visualize the results using Grad-CAM for explainability.

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf

# Add parent directory to path for imports
sys.path.append(os.path.dirname(os.getcwd()))

from models.covid_model import build_covid_model
from utils.data_loader import load_covid_data, segment_lungs
from utils.visualization import plot_gradcam_comparison
from utils.gradcam import compute_gradcam, overlay_gradcam

## Load Trained Model

First, we'll load the pre-trained COVID-19 detection model. If you don't have a trained model yet, you can run the training script from the scripts directory.

In [ ]:
# Path to the model (update based on your model location)
model_path = "../models/saved/covid_model.h5"

# Try to load the saved model, or build a new one if the file doesn't exist
try:
    model = tf.keras.models.load_model(model_path)
    print("Loaded pre-trained model from disk")
except:
    print("Pre-trained model not found, building a new one")
    model = build_covid_model(input_shape=(224, 224, 1))

# Display model summary
model.summary()

## Helper Functions for Inference

These functions will help us preprocess images and interpret model results.

In [ ]:
def preprocess_image(image_path, target_size=(224, 224)):
    """Preprocess a chest X-ray image for COVID-19 detection."""
    # Read image
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError(f"Could not read image: {image_path}")
    
    # Resize
    img = cv2.resize(img, target_size)
    
    # Apply CLAHE for contrast enhancement
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    img = clahe.apply(img)
    
    # Apply lung segmentation
    img_segmented = segment_lungs(img)
    
    # Normalize and add channel dimension
    img_normalized = img_segmented / 255.0
    img_normalized = np.expand_dims(img_normalized, axis=-1)
    
    return img, img_normalized

def interpret_prediction(prediction):
    """Interpret prediction probability for COVID-19 detection."""
    probability = prediction[0][0]
    prediction_class = "COVID-19 Positive" if probability > 0.5 else "Normal"
    confidence = probability if probability > 0.5 else 1 - probability
    
    return prediction_class, confidence

## Single Image Inference

Let's run inference on a single image and visualize the results.

In [ ]:
# Path to an example chest X-ray image (update with your own file)
example_image_path = "../data/covid/covid/example.jpg"

# Check if the file exists, otherwise use a placeholder
if not os.path.exists(example_image_path):
    # This is just for demonstration when you don't have the dataset
    print("Example image not found. Please update with a valid path to a chest X-ray image.")
    # Create a placeholder image
    img = np.random.rand(224, 224) * 255
    img = img.astype(np.uint8)
    img_normalized = img / 255.0
    img_normalized = np.expand_dims(img_normalized, axis=-1)
else:
    # Preprocess the image
    img, img_normalized = preprocess_image(example_image_path)

# Make prediction
prediction = model.predict(np.expand_dims(img_normalized, axis=0))
prediction_class, confidence = interpret_prediction(prediction)

# Print prediction
print(f"Prediction: {prediction_class}")
print(f"Confidence: {confidence:.2%}")

# Visualize original image and Grad-CAM
fig = plot_gradcam_comparison(
    model, 
    img_normalized, 
    title=f"COVID-19 Detection: {prediction_class} (Confidence: {confidence:.2%})"
)

## Batch Inference

Now let's run inference on multiple images from a directory.

In [ ]:
def process_directory(directory_path, num_samples=5):
    """Process multiple images from a directory."""
    results = []
    
    # Get all image files in the directory
    image_files = [f for f in os.listdir(directory_path) 
                   if os.path.isfile(os.path.join(directory_path, f)) 
                   and f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    # Limit to the specified number of samples
    image_files = image_files[:num_samples]
    
    for filename in image_files:
        image_path = os.path.join(directory_path, filename)
        img, img_normalized = preprocess_image(image_path)
        
        # Make prediction
        prediction = model.predict(np.expand_dims(img_normalized, axis=0))
        prediction_class, confidence = interpret_prediction(prediction)
        
        # Compute Grad-CAM
        heatmap = compute_gradcam(model, img_normalized)
        overlay = overlay_gradcam(img, heatmap)
        
        results.append({
            'filename': filename,
            'prediction': prediction_class,
            'confidence': confidence,
            'image': img,
            'overlay': overlay
        })
    
    return results

In [ ]:
# Path to a directory of chest X-ray images (update with your own directory)
test_directory = "../data/covid/covid"

# Process images if the directory exists
if os.path.exists(test_directory):
    results = process_directory(test_directory)
    
    # Visualize results
    fig, axes = plt.subplots(len(results), 2, figsize=(12, 4*len(results)))
    
    for i, result in enumerate(results):
        # Display original image
        axes[i, 0].imshow(result['image'], cmap='gray')
        axes[i, 0].set_title(f"{result['filename']}")
        axes[i, 0].axis('off')
        
        # Display Grad-CAM overlay
        axes[i, 1].imshow(result['overlay'])
        axes[i, 1].set_title(f"{result['prediction']} (Confidence: {result['confidence']:.2%})")
        axes[i, 1].axis('off')
    
    plt.tight_layout()
    plt.show()
else:
    print(f"Directory not found: {test_directory}")
    print("Please update with a valid directory path containing chest X-ray images.")

## Upload and Analyze Your Own Image

The following code allows you to upload your own chest X-ray image for analysis.

In [ ]:
from IPython.display import display, FileUpload

def process_uploaded_image(change):
    """Process an uploaded image for COVID-19 detection."""
    # Get the uploaded file
    uploaded_file = list(change['new'].values())[0]
    content = uploaded_file['content']
    filename = uploaded_file['name']
    
    # Save the file to a temporary location
    temp_path = f"temp_{filename}"
    with open(temp_path, 'wb') as f:
        f.write(content)
    
    # Preprocess the image
    img, img_normalized = preprocess_image(temp_path)
    
    # Make prediction
    prediction = model.predict(np.expand_dims(img_normalized, axis=0))
    prediction_class, confidence = interpret_prediction(prediction)
    
    # Visualize original image and Grad-CAM
    fig = plot_gradcam_comparison(
        model, 
        img_normalized, 
        title=f"COVID-19 Detection: {prediction_class} (Confidence: {confidence:.2%})"
    )
    
    # Clean up the temporary file
    os.remove(temp_path)
    
    return fig

# Create upload widget
uploader = FileUpload(accept='.jpg,.jpeg,.png', multiple=False)
uploader.observe(process_uploaded_image, names='value')

# Display the widget
print("Upload a chest X-ray image:")
display(uploader)

## Conclusion

In this notebook, we have demonstrated how to use the Doctor Dock COVID-19 detection model to:

1. Load and preprocess chest X-ray images
2. Make predictions to classify images as COVID-19 positive or normal
3. Visualize the model's decision-making process using Grad-CAM
4. Run batch inference on multiple images
5. Upload and analyze your own images

The Grad-CAM visualizations help to understand which regions of the X-ray the model is focusing on to make its predictions, providing explainability that is crucial for medical AI applications.